In [1]:
import pandas as pd
import pathlib as path

In [2]:
rel_path = path.Path().absolute()
od = pd.read_csv(str(rel_path) + r"\raw_data\Oscars-demographics-DFE_utf_8.csv")
ids = pd.read_csv(str(rel_path) + r"\raw_data\dfe_movieID.tsv", sep = '\t')
titles = pd.read_csv(str(rel_path) + r"\..\src\titles_during_oscars.csv")
ratings = pd.read_csv(str(rel_path) + r"\raw_data\title_ratings.tsv", sep = '\t')

del ids['Unnamed: 0']
del od['_trusted_judgments']
del od['_last_judgment_at']

In [3]:
ids_drop = ids.drop_duplicates()
df = od.merge(ids_drop, on = ['movie', 'year_of_award', 'person'])
del df['_unit_id']
del df['_golden']
del df['_unit_state']
del df['date_of_birth:confidence']
del df['race_ethnicity_gold']
del df['religion_gold']
del df['sexual_orientation_gold']
del df['year_of_award_gold']
del df['race_ethnicity:confidence']
del df['sexual_orientation:confidence']
del df['date_of_birth_gold']
del df['birthplace_gold']
del df['birthplace:confidence']
del df['religion:confidence']
del df['year_of_award:confidence']
df['movie_IMDB_Link'] = df['movieID']
df['movieID'] = df['movieID'].str.split('/', expand = True)[4]

In [4]:
t_df = df.merge(titles, left_on = 'movieID', right_on = 'tconst', how = "left")
del t_df["Unnamed: 0"]

In [72]:
mdf = t_df.merge(ratings, left_on = 'movieID', right_on = 'tconst')
mdf = mdf.replace('Na', 'Unknown')
mdf['religion'] = mdf['religion'].replace(['Roman Catholic',
             'Born-Again Christian',
             'Christian Science',
             'Anglican/episcopalian',
             'Baptist',
             'Disciples of Christ',
             'Lutheran', 'Presbyterian', 'Protestant', 'Quaker', 'Methodist', 'Congregationalist'
            ], 'Christian')
mdf['religion'] = mdf['religion'].replace('See Note', 'Unknown')
del mdf['tconst_x']
del mdf['titleType']
del mdf['primaryTitle']
del mdf['originalTitle']
del mdf['tconst_y']
del mdf['startYear']

mdf['rating_bin'] = mdf['averageRating'].round(0)

def custom_round(x, base=5):
    return int(base * round(float(x)/base))


mdf['runtime_bin'] = mdf['runtimeMinutes'].fillna(0)
mdf['runtime_bin'] = mdf['runtime_bin'].apply(lambda x: custom_round(x, base=30))/60

del mdf['averageRating']
del mdf['runtimeMinutes']
del mdf['movieID']
mdf

,birthplace,date_of_birth,race_ethnicity,religion,sexual_orientation,year_of_award,award,biourl,movie,person,bioLink,movie_IMDB_Link,rating_bin,runtime_bin
0,"Chisinau, Moldova",30-Sep-1895,White,Unknown,Straight,1927,Best Director,http://www.nndb.com/people/320/000043191/,Two Arabian Knights,Lewis Milestone,https://www.imdb.com/name/nm0587277/,https://www.imdb.com/title/tt0018515/,7.0,0.0
1,"Glasgow, Scotland",2-Feb-1886,White,Unknown,Straight,1930,Best Director,http://www.nndb.com/people/626/000042500/,The Divine Lady,Frank Lloyd,https://www.imdb.com/name/nm0515979/,https://www.imdb.com/title/tt0019824/,6.0,0.0
2,"Chisinau, Moldova",30-Sep-1895,White,Unknown,Straight,1931,Best Director,http://www.nndb.com/people/320/000043191/,All Quiet on the Western Front,Lewis Milestone,https://www.imdb.com/name/nm0587277/,https://www.imdb.com/title/tt0020629/,8.0,2.5
3,"Chicago, Il",23-Feb-1899,White,Unknown,Straight,1932,Best Director,http://www.nndb.com/people/544/000041421/,Skippy,Norman Taurog,https://www.imdb.com/name/nm0851537/,https://www.imdb.com/title/tt0022397/,6.0,1.5
4,"Salt Lake City, Ut",23-Apr-1894,White,Christian,Straight,1933,Best Director,http://www.nndb.com/people/292/000044160/,Bad Girl,Frank Borzage,https://www.imdb.com/name/nm0097648/,https://www.imdb.com/title/tt0021635/,6.0,1.5
5,"Glasgow, Scotland",2-Feb-1886,White,Unknown,Straight,1934,Best Director,http://www.nndb.com/people/626/000042500/,Cavalcade,Frank Lloyd,https://www.imdb.com/name/nm0515979/,https://www.imdb.com/title/tt0023876/,6.0,2.0
6,"Bisacquino, Sicily, Italy",18-May-1897,White,Christian,Straight,1935,Best Director,http://www.nndb.com/people/459/000032363/,It Happened One Night,Frank Capra,https://www.imdb.com/name/nm0001008/awards,https://www.imdb.com/title/tt0025316/,8.0,2.0
7,"Cadiz, Oh",1-Feb-1901,White,Unknown,Straight,1935,Best Actor,http://www.nndb.com/people/156/000027075/,It Happened One Night,Clark Gable,https://www.imdb.com/name/nm0000022/awards,https://www.imdb.com/title/tt0025316/,8.0,2.0
8,"Paris, France",13-Sep-1903,White,Unknown,Straight,1935,Best Actress,http://www.nndb.com/people/935/000043806/,It Happened One Night,Claudette Colbert,https://www.imdb.com/name/nm0001055/awards,https://www.imdb.com/title/tt0025316/,8.0,2.0
9,"Cape Elizabeth, Me",1-Feb-1894,White,Christian,Bisexual,1936,Best Director,http://www.nndb.com/people/458/000032362/,The Informer,John Ford,https://www.imdb.com/name/nm0000406/awards,https://www.imdb.com/title/tt0026529/,8.0,1.5


In [76]:
mdf.to_csv(str(rel_path) + r'\..\src\oscar_demos_def.csv')